# Testing for Differences between Groups of Edges

Let's recall back to our school example. We have a network consisting of $100$ students who attend one of two schools, with the first $50$ students attending school one and the second $50$ students attending school two. The nodes are students, and the edges are whether a pair of students attend the same school. What we want to know is whether there is a higher chance two students are friends if they go to the same school than if they go to two different schools. We don't know which student goes to which school, so we have some work to do. The network is given to us looking something like this:

In [ ]:
from graspologic.simulations import sbm
import numpy as np

# define the parameters for the SBM
ns = [50, 50]
n = np.array(ns).sum()
B = [[0.6, 0.3],[0.3, 0.6]]
ys = [1 for i in range(0, ns[0])] + [2 for i in range(0, ns[1])]

A = sbm(n=[50, 50], p=B, loops=False, directed=False)

# generate a reordering of the n nodes
vtx_perm = np.random.choice(n, size=n, replace=False)
Aperm = A[tuple([vtx_perm])] [:,vtx_perm]
yperm = np.array(ys)[vtx_perm]

In [ ]:
from graphbook_code import draw_multiplot
import matplotlib.pyplot as plt

draw_multiplot(Aperm, title="School Network");

The true, but *unknown*, probability matrix looks something like this:

In [ ]:
z = np.array([[1, 0] for i in range(0, ns[0])] + [[0, 1] for i in range(0, ns[1])])
P = z @ B @ z.T

Pperm = P[tuple([vtx_perm])] [:,vtx_perm]

In [ ]:
from graspologic.plot import heatmap

ax = heatmap(Pperm, vmin=0, vmax=1, title='True Probability Matrix, School Network')

How do we proceed? 

We begin by using spectral methods whichh were developed in [Chapter 6](#link?). Specifically, here we will use the adjacency spectral embedding, and estimate the number of embedding dimensions:

In [ ]:
from graspologic.embed import AdjacencySpectralEmbed

ase = AdjacencySpectralEmbed()  # adjacency spectral embedding, with optimal number of latent dimensions selected using elbow picking
Xhat = ase.fit_transform(Aperm)

We visualize the estimated latent positions using a pairs plot:

In [ ]:
from graspologic.plot import pairplot

_ = pairplot(Xhat, title="Pairs plot for school network")

It seems pretty obvious that thhere are two distinct communities indicated by the latent positions, which makes sense given that there are students from two schools present. This is indicated in two off-diagonal plots, as we can see that in the upper-right plot for instance, the second latent dimension separates two clusters of latent positions approximately right down the middle. `K-means` picks up on this trend, too:

In [ ]:
from graspologic.cluster import KMeansCluster

km_clust = KMeansCluster(max_clusters = 10)
km_clust = km_clust.fit(Xhat);

We start by visualizing the number of clusters produced by `K-means`, using the silhouette score:

In [ ]:
nclusters = range(2, 11)  # graspologic nclusters goes from 2 to max_clusters
silhouette = km_clust.silhouette_  # obtain the respective silhouette scores

In [ ]:
import seaborn as sns
from pandas import DataFrame as df

silhouette_df = df({"Number of Clusters": nclusters, "Silhouette Score": silhouette})  # place into pandas dataframe

fig, ax = plt.subplots(1,1,figsize=(8, 4))

sns.lineplot(data=silhouette_df,ax=ax, x="Number of Clusters", y="Silhouette Score");
ax.set_title("Silhouette Analysis of KMeans Clusterings")
fig;

It i clear that the silhouette score is maximized for $K=2$ clusters, so `K-means` has indicated that there are two clusters in the latent positions, and consequently two communities of nodes. Next, we produce community estimates for each of the nodes according to their latent positions. Further, since this example is a simulated network, we can also re-align the predicted labels with the true labels. Finally, we computer the overall quality of the clustering using the Adjusted Rand Index, and we visualize the node predictions:

In [ ]:
from graspologic.utils import remap_labels
from sklearn.metrics import adjusted_rand_score

labels_autokmeans = km_clust.fit_predict(Xhat)
labels_autokmeans = remap_labels(yperm, labels_autokmeans)

ari_kmeans = adjusted_rand_score(labels_autokmeans, yperm)

In [ ]:
pairplot(Xhat,
         labels=labels_autokmeans,
         title=f'KMeans on embedding, ARI: {str(ari_kmeans)[:5]}',
         legend_name='Predicted label',
         height=3.5,
         palette='muted',);

Since the ARI is $1.0$, `K-means` has perfectly predicted which community each node is assigned to. Finally, let's take the original adjacency matrix, and just resort the nodes based on the predicted node communities:

In [ ]:
# use argsort to sort by the kmeans labels
node_order = np.argsort(labels_autokmeans)

# resort the adjacency matrix
Asrt = Aperm[tuple([node_order])][:,node_order]
ysrt = labels_autokmeans[node_order]

In [ ]:
draw_multiplot(Asrt, labels=ysrt, title="School Network, nodes sorted by predicted community");

So, our question of interest is whether the within-school connectivity exceeds the between-school connectivity. Let's think of how to better formulate this problem.

Everything we have done up until this point in the book treated *nodes* as the fundamental objects of interest for statistical modelling. We have communities of nodes, and clusters of latent positions corresponding to nodes which comprise these communities. 

Now, we will introduce a model which instead focuses on clusters of *edges*. Notice that the edges amongst students of school one are all in the upper left square of the adjacency matrix, and the edges amongst students of school two are all in the lower right square of the adjacency matrix. We will call these edges the "within-school" edges (Cluster $1$). The edges between students of school one and two are in the upper right and upper left squares of the adjacency matrix (Cluster $2$). In a picture, the edges are grouped as follows:

In [ ]:
Z = np.zeros((n,n))

Z[0:50,0:50] = 1  # within-school edge
Z[0:50,50:100] = 2  # between-school edge
Z[50:100,50:100] = 1  # within-school edge
Z[50:100,0:50] = 2  # between-school edge

In [ ]:
from graphbook_code import heatmap

fig, ax = plt.subplots(1,2,figsize=(15, 6))

heatmap(Asrt, ax=ax[0], title='School Network, sorted')
heatmap(Z, ax=ax[1], color="qualitative", title="Edge Cluster");

So now, we know which cluster each edge is in, and we have a sample of a network. Recall that our original question was whether the indivduals within-school (cluster $1$) were better friends than individuals between-schools (cluster $2$). Reformulating this question in terms of the network, our question becomes whether the edges which are assigned to cluster $1$ have a *higher* probability of existing than the edges of cluster $2$. For this purpose, we introduce a *different* generalization of the Stochastic Block Model, the *Structured Independent Edge Model* (SIEM).

## The Structured Independent Edge Model is parametrized by a Cluster-Assignment Matrix and a probability vector

To describe the Structured Independent Edge Model (SIEM), we will return to our old coin flipping example.

### The Cluster-Assignment Matrix

The cluster assignment matrix $Z$ is an $n \times n$ matrix which assigns potential edges in the random network to clusters. What do we mean by this?

Remember that the adjacency matrix $\mathbf A$ for a random network is *also* an $n \times n$ matrix, where each entry $\mathbf a_{ij}$ is a random variable which takes the value $0$ or the value $1$ with a particular probability. The cluster assignment matrix takes each of these $n^2$ random variables, and uses a parameter $z_{ij}$ to indicate which of $K$ possible clusters this edge is part of. In the school example, for instance, the edges in the upper left and lower right are in cluster $1$ ('within-school' edges), and the edges in the upper right and lower left are in cluster $2$ ('between-school' edges). In the school example, $K=2$.

### The Probability vector

the second parameter for the SIEM is a probability vector, $\vec p$. If there are $K$ edge clusters in the SIEM, then $\vec p$ is a length-$K$ vector. Each entry $p_k$ indicates the probablity of an edge in the $k^{th}$ cluster existing in the network. For example, $p_1$ indicates the probability of an edge in the first edge cluster, $p_2$ indicates the probability of an edge in the second edge cluster, so on and so-forth. In the school example, for instance, the probability $p_1$ indicates the probability of a within-school edge, and the probability $p_2$ indicates the probability of a between-school edge.

Like usual, we will formulate the SIEM using our old coin flip example. We begin by obtaining $K$ coins, where the $k^{th}$ coin has a chance of landing on heads of $p_k$, and a chance of landing on tails of $1 - p_k$. For each entry $\mathbf a_{ij}$, we identify the corresponding cluster $z_{ij}$ that this edge is in. Remember that $z_{ij}$ takes one of $K$ possible values. We flip the $z_{ij}$ coin, and if it lands on heads (with probability $p_{z_{ij}}$), the edge exists, and if it lands on tails (with probability $1 - p_{z_{ij}}$) the edge does not exist.

If $\mathbf A$ is an SIEM random network with $n$ nodes, the cluster assignment matrix $Z$, and the probability vector $\vec p$, we say that $\mathbf A$ is an $SIEM_n(Z, \vec p)$ random network.

### How do we simulate samples from an $SIEM_n(Z, \vec p)$ random network?

The procedure below will produce for us a network $A$, which has nodes and edges, where the underlying random network $\mathbf A$ is an $SIEM_n(Z, \vec p)$ random network:

```{admonition} Simulating a sample from an $SIEM_n(Z, \vec p)$ random network
1. Determine a cluster assignment matrix $Z$ for each edge of the network. Each edge entry $z_{ij}$ can take one of $K$ possible values.
2. Determine a length-$K$ probability vector for each of the $K$ edge clusters.
3. For each of the $K$ clusters, obtain $K$ total weighted coins, where the $k^{th}$ coin lands on heads with probability $p_k$ and tails with probability $1 - p_k$.
4. For each edge $(i, j)$:
    * Denote $z_{ij}$ to be the cluster assignment of the potential edge between node $i$ and node $j$.
    * Flip the $z_{ij}$ coin, and if it lands on heads, the corresponding entry in the adjacency matrix $a_{ij}$ is $1$. If it lands on tails, the coresponding entry in the adjacency matrix $a_{ij}$ is $0$. 
5. The adjacency matrix we produce, $A$, is a sample of an $SIEM_n(Z, \vec p)$ random network.
```

## Why do we care about the SIEM?

If you recall from the [Section on RDPGs](#link?), we learned that for every $SBM_n(\vec z, B)$ random network, there is a corresponding latent position matrix $X$ such that the probability matrices of the $SBM_n(\vec z, B)$ random network and an $RDPG_n(X)$ random network are the same. In this sense, the RDPG random networks *generalize* the SBM random networks, in that all SBM random networks are RDPG random networks, but not all RDPG random networks are SBM random networks. We can find latent position matrices which produce networks which cannot be summarized with an SBM, such as the [later example we saw in the RDPG section](#link?). The SIEM has a similar relationship with the SBM: all SBM random networks are SIEM random networks, but not all SIEM random networks are SBM random networks. If you want the gorey details of how this works, check out the block below. If not, you can skip on to the next paragraph.

```{admonition} All SBMs are SIEMs
What this means is that for any $SBM_n(\vec z', B)$ random network, there is a corresponding $Z$ and $\vec p$ such that an $SIEM_n(Z, \vec p)$ random network has the same probability matrix. We can see this pretty easily: if $K$ was the number of communities of the SBM, we can define an SIEM to have a single edge cluster for each *pair* of communities. We can see this as follows:
1. Define the number of edge clusters: Edge cluster $1$ consists of edges between nodes both in community $1$, edge cluster $2$ consists of edges between a node in community $1$ and a node in community $2$, so on and so forth up to edges between a node in community $1$ and a node in community $K$. Edge cluster $K+1$ consists of edges between a node in community $2$ and a node in community $1$, and again we repeat this up to edge cluster $2K$ which delineates edges between a node in community $2$ and a node in community $K$. We continue this procedure up to edge cluster $K^2$ which is between two nodes in community $K$. This gives us a map, where for each pair of communities $l$ and $l'$, we have an edge cluster $k$ whch corresponds to this pair of communities.
2. Define the edge cluster matrix: Proceed edge-by-edge through the network looking at the community asignment vector $\vec z'$. The edge cluster for an edge $(i, j)$ is the edge cluster we identified above between the communities for node $i$ and node $j$, $z_i'$ and $z_j'$ respectively.
3. Define the edge cluster probability vector: Let the probability vector $\vec p$ be defined such that, iff the $l$ and $l'$ communities correspond to edge cluster $k$, then $p_k$ has the same value as $b_{ll'}$ in the block matrix.
This produces an $SIEM_n(Z, \vec p)$ random network with the same probability matrix as an $SBM_n(\vec z, B)$ random network.
```

For this reason, we will develop statistical tools for the SIEM, but it is important to note that they apply equally well to an SBM random network.

Remember that our question boiled down to whether the edges which are assigned to cluster $1$ have a *higher* probability of existing than the edges of cluster $2$. In other words, we *hypothesize* that the edges assigned to cluster $1$ have a higher probability of existing than the edges of cluster $2$. In terms of the probability vector, what we are saying is that we think that $p_1 > p_2$. Obviously, we are wrong if $p_1 \leq p_2$. This type of statistical question is known as a *hypothesis test*. **Hypothesis testing** is an approach by which we formulate statistical questions and present approaches which, *using the data*, indicates how much the data supports or doess not support the statistical questions we asked (or, provide us with the ability to *infer* something). For this reason, hypothesis testing is often an essential component of *statistical inference*, which is the name given to the process of learning something from data using statistical models. 

The way hypothesis testing works is we call one hypothesis a *null hypothesis* and the other an *alternative hypothesis*. The **null hypothesis** is the hypothesis where, if true, indicates that we were *wrong* about what how we thought the system behaved. In our example data, the null hypothesis is that $p_1 \leq p_2$. We typically would write this out notationally as $H_0: p_1 \leq p_2$. The capital $H$ is used to denote that this is a hypothesis, and the little "$0$" is just to indicate "null". The **alternative hypothesis** is the hypothesis where, if true, indicates that we werer *correct* about how we thought the system behaved. In our example, the alternative hypothesis is that $p_1 > p_2$, which we write out as $H_A: p_1 > p_2$. Here, the subscript $A$ denotes that this is the alternative hypothesis. When describing our hypothesis, we say that we are testing $H_A: p_1 > p_2$ *against* $H_0: p_1 \leq p_2$. This is because a hypothesis test must specifically delineate the possible situations in which we are *right* (the alternative $H_A$) and when we are *wrong* (the null $H_0$). We perform statisical inference by looking at how well the data we are presented with supports, or reflects, the alternative hypothesis.

### Hypothesis Testing with coin flips

To describe how hypothesis testing works from an implementation perspective, we'll naturally return to coin flips. If you remember way back to the [Preface of Chapter 5](#link?), we introduced a game. The game was as follows: a gambler has a coin, and asks us to gamble one dollar. If the coin lands on heads, we obtain our dollar back, and an additional dollar. If the coin lands on tails, we lose our dollar. We get to observe $10$ random coin flips, and determine whether we want to play the game. 

As it turns out, if this coin has a probability of landing on heads greater than $0.5$, we will, over time, *make* money if we continue to play the game. If the coin has a probablity off landing on heads lesss than $0.5$, we will, over time, *lose* money if we continue to play the game. So, if we are going to play the game in a principled way, we want to be really sure that we are going to gain money by playing. How can we use hypothesis testing to use a little bit of math to determine whether we should play or not?

To begin, we need to delineate our hypotheses. Here, our question of interest is, should we play the game? Therefore, the alternative hypothesis is the one where we *should* play the game, or the coin has a probability greater than $0.5$. The null hypothesis is that we should *not* play the game, or the coin has a probability that is not greaterr than $0.5$. If we use $p$ to denote the probability that the gambler's coin lands on heads, we write this as $H_A: p > 0.5$, against the null hypothesis that $H_0: p \leq 0.5$.

Let's say that we observe $10$ coin flips, and we see that the coin lands on heads $6$ times. If we were to estimate the coin's probability of landing on heads, like we did in the [Preface of the Section on MLE](#link?), we would get that $\hat p = \frac{6}{10}$, or that our estimate $\hat p = 0.6$. It's pretty obvious that $\hat p$ is greater than $0.5$, so we should play the game, right?

The answer is a little more complicated than that. If you take a fair coin (one that is roughly equally likely to land on heads and tails) and flip it enough times, you will probably find a sequence of ten flips with six heads. We are scientists, after all, so we want to be *really* sure that if we play we are going to make money. In statistical speak, we want to be sure that if we play the game, there is a low chance that we are actually wrong. We want to know exactly what the chances are that, if we were *wrong*, we could still have seen the estimate $\hat p$ that we calculated. This is where a concept called a *p-value* comes into play. 

#### $p$-values tell us the chances of observing an outcome if the null hypothesis is true

To understand the $p$-value, we must first introduce a concept called a *test statistic*. A test-statistic is a fancy word for a data-derived quantity that summarizes our data with a single number which will be used for hypothesis testing. The test statistic should capture a property of the data that is relevant to our question of interest (here, the probability that the coin lands on heads), so it makes sense in our example to just use the estimate of the probability that the coin lands on heads as our test statistic. In general, the **$p$-value** is the probability that we would incorrectly reject the null hypothesis (that is, that the coin is fair) in favor of the alternative hypothesis (that is, that the coin is not fair) if the null hypothesis were really true. 

Next, we need to think about what types of values we would see from the test statistic if the null hypothesis were true. If the coin really had a probability of $0.5$ of landing on heads and we flipped it $10$ times, there is a non-zero probabaility that we would see all heads and our estimate would have been $1.0$, that we would see $9$ heads and $1$ tails and our estimate would have been $0.9$, so on and so-forth all the way to the outcome where we see only tails and our estimate would have been $0.0$. Which one we observed when watching the gambler flip the coin would come down to *random chance*, so we will use the random variable $\mathbf{\hat p}_0$ to describe this quantity. Specifically, $\mathbf{\hat p}_0$ is a random variable which takes values between $0$ and $1$ in $0.1$ probability increments. $\mathbf{\hat p}_0$'s probability of taking a given increment $k$ is equal to the probability that we would have estimated the probability to be $k$ if the null hypothesis (the coin is fair and lands on heads with probability $0.5$) were true. $\mathbf{\hat p}_0$ takes each increment with the following probabilities:

In [ ]:
import seaborn as sns
import pandas as pd
from scipy.stats import binom
import warnings
warnings.filterwarnings("ignore")

incr = [i/10 for i in range(0, 11)]
probs = [binom.pmf(k=i*10, n=10, p=0.5) for i in incr]

fig, ax = plt.subplots(1,1, figsize=(10, 5))
sns.barplot(incr, probs, color="blue", ax=ax);
ax.set_xlabel("Estimate of probability of landing on heads, $\hat p_0$")
ax.set_ylabel("Probability of seeing this estimate if $H_0$ true")
ax.set_title("Distribution of $\hat p_0$");
ax.axvline(6, color="red")
ax.text(6.5, .2, "Estimate we observed, $\hat p = 0.6$", color="red");

So, this is a little surprising! Even if the null hypothesis were true and the coin really landed on heads with a probability of $0.5$, there is still a pretty good chance we would observe a probability of at least $0.6$ if we were to flip a fair coin $10$ times. We describe this quantity using a $p$-value, which is the probability of seeing a test statistic at least as extreme as the one we calculated from our data if the null hypothesis were true. Here, the $p$-value is the probability that, if the coin were fair, we would produce an estimate of $0.6$, $0.7$, $0.8$, $0.9$, or $1.0$. This is the sum of all of the heights of the corresponding bars in the above plot, which turns out to be $0.377$. This approach is called the *binomial test*, which we can implement in python as follows:

In [ ]:
from scipy.stats import binom_test

nheads = 6  # the number of heads in the experiment
nflips = 10  # the total number of coin flips in the experiment
p0 = 0.5  # the probability of the coin under the null hypothesis

print("p-value: {:.3f}".format(binom_test(nheads, nflips, p=p0, alternative="greater")))

This means that, even if the coin were fair, we still would have produced an estimate of $0.6$ about $37.7\%$ of the time, and we would be *wrong* to play the game. This might be enough for you to play the game, but as scientists we usually like to be a little more confident than this when we report something. We use the term the "$\alpha$ of the test" to describe what an acceptable chance of being incorrect is for us. We can pick $\alpha$ to be anything we want really so long as decide this threshold before calculating the $p$-value. Usually, scientists choose $0.05$, which means that we want there to be less than a $5\%$ chance that we are wrong before making a decision. In other words, we look for a $p$-value that is less than $0.05$. 

#### Two-sample hypothesis testing with coins

Let's say after seeing the first game, you determine that a $37.7\%$ chance of being wrong is a little too risky for you. The gambler presents you with a new game. He has two coins this time, and will even let you pick which to use. You gamble one dollar, and if your coin comes up heads but his comes up tails, you get your dollar plus an additional dollar. If both coins land on heads or tails, you get your dollar back. If his coin comes up heads but your coin comes up tails, you lose your dollar. This time, you get to see the gambler flip each coin ten times before determining whether you want to play. The first coin lands on heads seven times, and the second coin lands on heads only once. If the first coin actually has a higher chance of landing on heads than his coin, you should play the game. Will you gamble this time?

To formalize this situation, we'll use similar notation to before. We'll call $p_1$ the probability that your coin lands on heads, and $p_2$ the probability that his coin lands on heads. Our alternative hypothesis here is $H_A: p_1 \neq p_2$, against the null hypothesis $H_0: p_1 = p_2$. A **two-sample test** is a test which is performed when we have two random samples, and we want to investigate whether there is a difference between the two random samples. Specifically here, we have the outcomes of coin flips from each of the two coins, and we want to determine whether the first coin is actually different from the second coin. Intuitionally, the idea is that even if the first coin did not have a higher probability of landing on heads than the second coin, there is still some probability that if we flipped each coin ten times, we would get more heads with the second coin. We see that our estimates of the probabailities are $\hat p_1 = \frac{7}{10} = 0.7$ and $\hat p_2 = \frac{1}{10} = 0.1$ respectively. 

When we have two samples of data whose outcomes boil down to ones and zeros (or, heads and tails), it turns out that the intuition gets more complex pretty quickly. For this reason, we won't explain the two-sample test for binary data in quite as muc depth as we did with the previous example (the **one-sample test**), but much of the same intuition holds. Again, we seek to compute a $p$-value which tells us the chances of observing that $\hat p_1 = 0.7$ and $\hat p_2 = 0.1$ if $p_1$ weren't actually greater than $p_2$. In this case, the appropriate statistical test is known as [Fisher's Exact Test](https://en.wikipedia.org/wiki/Fisher%27s_exact_test), which we can implement using `scipy`. The inputs for Fisher's exact test is a contingency table of the results:

| | First coin | Second coin |
|---|---|---|
| Landed on heads | 7 | 1 |
| Landed on tails | 3 | 9 |

In [ ]:
from scipy.stats import fisher_exact

table = [[7,3], [1,9]]
test_statistic, pval = fisher_exact(table)
print("p-value: {:.3f}".format(pval))

This means that, there is only about a $2\%$ chance that the two coins have the same probability of landing on heads, as the $p$-value is $0.02$. If we were to again use $\alpha=0.05$ as our decision threshold, this means we should play this game as many times as the gambler will allow before he catches on. Next, we'll see how this strategy of coin flipping applies to the SIEM.

### Hypothesis Testing using the SIEM

#### Unweighted networks

To shift back to our network, we had the following network sample and edge cluster matrix:

In [ ]:
from graphbook_code import heatmap, cmaps

fig, ax = plt.subplots(1,2,figsize=(15, 6))

heatmap(Asrt, ax=ax[0], title='School Network, sorted')
heatmap(Z, ax=ax[1], color="qualitative", title="Edge Cluster");

The probability vector for the SIEM is estimated very similar to how we used maximum likelihood estimation for the SBM, in that we just take the fraction of edges which exist in the network for each edge cluster:

In [ ]:
p = [np.sum(Asrt[Z == i])/np.size(Asrt[Z == i]) for i in [1, 2]]
[print("Probability for cluster {}, p{}: {:.3f}".format(i, i, p[i - 1])) for i in [1, 2]];

These estimates are pretty different, but the core question we want to answer is, how confident are we that this discrepancy between the estimated probabilities is not due to random chance?

It turns out that this question is effectively the same question as we asked for the coin flips. Specifically, we want to formulate the hypothesis test with $H_0: p_1 \leq p_2$ against $H_A: p_1 > p_2$. We proceed with exactly the same framework as we used for the coin tosses. We again construct a contingency table, with the following table margins:

| | Cluster 1, the within-school edges | Cluster 2, the between-school edges |
| --- | --- | --- |
| Number of edges | $a$ | $b$ |
| Number of non-edges | $c$ | $d$ |

The entry $a$ is the total number of edges in the first cluster, and $b$ is the total number of edge in the second cluster. The entry $c$ is the total number of possible edges in cluster one which do not exist (the number of adjacencies in cluster one with an adjacency of zero), and the entry $d$ is the total number of possible edges in the second cluster which do not exist (the number of adjacencies in cluster two with a value of zero). We can fill in the entries of this table using some basic `numpy` functions. Note that since this network is undirected and loopless, we want to look only at the edges which are in the upper triangle of the adjacency martix, and exclude the diagonal:

In [ ]:
# get the indices of the upper triangle of Asrt
upper_tri_idx = np.triu_indices(Asrt.shape[0], k=1)
# create a boolean array that is nxn
upper_tri_mask = np.zeros(Asrt.shape, dtype=bool)
# set indices which correspond to the upper triangle to True
upper_tri_mask[upper_tri_idx] = True

table = [[(Asrt[(Z == 1) & upper_tri_mask] == 1).sum(), np.sum(Asrt[(Z == 2) & upper_tri_mask] == 1)],
         [(Asrt[(Z == 1) & upper_tri_mask] == 0).sum(), np.sum(Asrt[(Z == 2) & upper_tri_mask] == 0)]]

In [ ]:
from graphbook_code import cmaps

fig, ax = plt.subplots(1,1,figsize=(8, 6))
sns.heatmap(np.array(table, dtype=int), ax=ax, square=True, cmap=cmaps["sequential"],
            annot=True, fmt="d", vmin=0, cbar_kws={"label": "Count"})
ax.set_xticks([0.5, 1.5])
ax.set_xticklabels(["Cluster 1", "Cluster 2"])
ax.set_yticks([0.5, 1.5])
ax.set_yticklabels(["Number of edges", "Number of non-edges"], rotation=0)
ax.set_title("Contingency Table, School Network");

We again use Fisher's exact test to test our hypothesis:

In [ ]:
test_statistic, pval = fisher_exact(table)
print("p-value: {:.3f}".format(pval))

And we end up with a $p$-value that is approximately zero. With our decision threshold still at $\alpha = 0.05$, since the $p$-value is less than $\alpha$, we have evidence to support that the alternative hypothesis is true. So, after a *lot* of build up spanning numerous sections, we have shown that the probability of two individuals being friends if they attend the same school is higher than the probability of two students being friends if they go to different schoools. Pretty exciting, huh?

#### Weighted Networks

When the networks were unweighted, everything made complete sense in the context of our coin flip regime. For weighted networks, on the other hand, things can go a *little* bit differently. To understand this question in the context of weighted networks, we'll return to our discussion on the SIEM, and reformulate it a little bit for the case where the adjacency matrix's entries take non-binary values (they aren't just zeros and ones).

##### The weighted SIEM has a vector of distribution functions

For the weighted SIEM, the first parameter is identical to that of the unweighted SIEM: the edge cluster matrix, $Z$, which is an $n \times n$ matrix whose entries $z_{ij}$ indicate which of the $K$ clusters the $(i, j)$ edge is in.

The second parameter is a little different. Remember for the unweighted case, we had a probability vector, $\vec p$, whose entries $p_k$ indicated the probability of an edge in the $k^{th}$ edge cluster. How do we extend this concept to edges which are not binary valued? To accomplish this, we introduce disribution functions.

Remember we said that, for a particular edge $\mathbf a_{ij}$ where the edge cluster for the $(i,j)$ edge was $z_{ij}$, that $\mathbf a_{ij}$ took a value of one with probability $p_{z_{ij}}$, and zero with probability $1 - p_{z_{ij}}$. As it turns out, what this says at a deeper level is that $\mathbf a_{ij}$ has a *distribution* whose parameter is just a probability $p_{z_{ij}}$. The distribution, in this case, is the *Bernoulli* distribution with probabilty $p_{z_{ij}}$, which we learned about in [the preface for chapter 5](#link?). However, there are other distributions we could use as well for $\mathbf a_{ij}$. 

You are probably already familiar with the Normal distribution, which has a mean $\mu$ and a standard deviation $\sigma$. What you need to know about the Normal distribution is that, if you were to sample a large number of points and then plotted their values on a histogram, you would see "roughly" a Bell-curve type of shape to the tops of the histogram bars. These bars would be centered roughly around the mean value $\mu$, the bars would be roughly symmetric (the heights on either side of the mean would be about equal), and that about $68\%$ of the data would be between $\mu - \sigma$ and $\mu + \sigma$ (within one standard deviation). In the below plot, we will simulate $10,000$ samples $x_i$ of random variables $\mathbf x_i$, which are Normally distributed with a mean of $0$ and a standard deviation of $5$, $10,000$ samples $y_i$ of random variables $\mathbf y_i$ with are Normally disributed with a mean of $5$ and a standard deviation of $5$, and $10,000$ samples $z_i$ of random variables $\mathbf z_i$ which are Normally distributed with a mean of $0$ and a standard deviation of $15$. We will then compute histograms so we can get a sense of how the Normal distribution behaves:

In [ ]:
import pandas as pd

params = {"xi": {"loc":0, "scale": 5}, "yi": {"loc":10, "scale": 5}, "zi": {"loc":0, "scale": 15}}
nsim = 50000

samples = []
for idx, param in params.items():
    samples.extend([{"Sample": idx, "Value": x} for x in np.random.normal(size=nsim, **param)])

real_df = pd.DataFrame(samples)

fig, ax = plt.subplots(1,1, figsize=(12, 6))
sns.histplot(real_df, x="Value", hue="Sample", stat="probability",
            ax=ax, common_norm=False)
ax.set_ylabel("Fraction of Points");

Notice that samples of $\mathbf x_i$ and $\mathbf y_i$ differ in the approximate centers of the histograms. This reflects that the mean of the random variables $\mathbf x_i$ are $10$ units less than the means of $\mathbf y_i$. Further, samples of $\mathbf z_i$ appear to take a "wider" range of values than samples of $\mathbf x_i$, despite the centers looking to be in approximately the same spot (right around zero). This reflect that the standard deviation of the random variables $\mathbf z_i$ exceed that of the $\mathbf x_i$ by $10$ units. We could, in a generic sense, say that the points $\mathbf x_i$ have distribution $F_x$, the points $\mathbf y_i$ have the distribution $F_y$, and the points $\mathbf z_i$ have the distribution $F_z$. Here, $F_x$ just so happens to be the Normal distribution with mean $0$ and standard deviation $5$, but we don't need to be that specific when defining the weighted SIEM.

For the weighted SIEM, we will have a length-$K$ vector $\vec F$ of distribution functions, where $F_k$ defines the distribution for the $k^{th}$ cluster. In this sense, $F_k$ could be any distribution really; it could be the normal distribution with mean $\mu_k$ and standard deviation $\sigma_k$, it could be the Bernoulli distribution with probability $p_k$, or it could be something more complicated. There are any number of possibilities for what the edge-weight distribution could be. 

Unfortunately, we can't quite use our coin flip example since the weighted SIEM does not necessarily have binary-valued adjacencies. For each entry $\mathbf a_{ij}$, we identify the corresponding cluster $z_{ij}$ that this edge is in. Remember that $z_{ij}$ takes one of $K$ possible values. We say that $\mathbf a_{ij}$ has the distribution indicated by the distribution function $F_{z_{ij}}$.

If $\mathbf A$ is an SIEM random network with $n$ nodes, the cluster assignment matrix $Z$, and the distribution vector $\vec F$, we say that $\mathbf A$ is an $wSIEM_n(Z, \vec F)$ random network.

Next, let's look at how one would sample from a weighted SIEM. The procedure below will simulate a sample $A$ of the random network $\mathbf A$ which is $wSIEM_n(Z, \vec F)$. 

```{admonition} Simulating a sample from an $wSIEM_n(Z, \vec F)$ random network
1. Determine a cluster assignment matrix $Z$ for each edge of the network. Each edge entry $z_{ij}$ can take one of $K$ possible values.
2. Determine a length-$K$ distribution vector for each of the $K$ clusters.
4. For each edge $(i, j)$:
    * Denote $z_{ij}$ to be the cluster assignment of the potential edge between node $i$ and node $j$.
    * Sample a single sample from the distribution function $F_{z_{ij}}$, and record the result as $\mathbf a_{ij}$. 
5. The adjacency matrix we produce, $A$, is a sample of an $SIEM_n(Z, \vec p)$ random network.
```

Next, let's come up with a working example for the weighted SIEM network. Let's imagine that we have the brain of an individual. The $n=100$ nodes are areas of the brain. For instance, there is a node of the brain responsible for movement, an area of the brain which is responsible for sight, so on and so forth. As you may be aware, the brain is split into two hemispheres, the left and right hemispheres. In many cases, these areas of the brain have a *bilateral analogue*, which means that the same area of the brain appears in the left and right hemispheres. While the region of the brain may have a slightly different function in each of the two hemispheres, they tend to work together to perform functions for the individual. For instance, if the left motor area is active when someone is walking, the right motor area will tend to be active too. In this case, our edge weights are how strongly two regions of the brain tend to *correlate* with one another over the course of the day. As we learned in the section on the [rho-correlated SBM], correlation is a value between $-1$ (the two areas are *never* active together) and $1$ (the two areas *always* are active together). If we organize the nodes first by hemisphere (left, then right) and second by the region of the brain, what we would anticipate is that the bilateral areas would tend to be higher correlated than non-bilateral areas. This would be reflected as an "off-diagonal" band (the *bilateral band*, cluster 2) that tends to have higher values than the other entries of the adjacency matrix (*non-bilateral edges*, cluster 1). The way we will simulate this is as follows. The non-bilateral edges will have a distribution that is between $-1$ and $1$, with no particular values being more or less likely. On the other hand, the bilateral band will tend to have higher values:

In [ ]:
from graphbook_code import siem

n = int(100)
p = [1, 1]  # edges are always present
Z = np.ones((n, n), dtype=int)  # initialize edge clusters Z
band_idx = np.where((np.eye(n, k=int(n/2)) == 1) | (np.eye(n, k=int(-n/2)) == 1))

Z[band_idx] = 2  # set the bilateral edges to cluster 2
Z[np.diag_indices(n)] = 0  # remove the diagonals from edge clusters, since the network is loopless

A = siem(n, p=p, edge_comm=Z, wt=[np.random.uniform, np.random.beta], wtargs=[{"low": -.5, "high": .5}, {"a": 8, "b": 2}])

In [ ]:

fig, ax = plt.subplots(1,2,figsize=(15, 6))
ys = ["Left" for i in range(0, int(n/2))] + ["Right" for i in range(0, int(n/2))]
heatmap(A, ax=ax[0], inner_hier_labels=ys, title='Brain Network')
heatmap(Z.astype(int), ax=ax[1], color="qualitative", title="Edge Cluster");

Notice that the edges in edge-cluster two (the bilateral bands) tend to have higher correlations than the edges in edge-cluster one (the non-bilateral bands). We can summarize this property by looking at a histogram of the two edge clusters:

In [ ]:
samples = []
edge_comm_name = {1: "Bilateral", 2: "Non-bilateral"}
for clust in [1, 2]:
    idx_clust = np.where(Z == clust)
    samples.extend([{"Cluster": "Cluster {}, {}".format(clust, edge_comm_name[clust]), "Correlation": A[x]} for x in zip(*idx_clust)])

real_df = pd.DataFrame(samples)

fig, ax = plt.subplots(1,1, figsize=(12, 6))
sns.histplot(real_df, x="Correlation", hue="Cluster", stat="probability",
            ax=ax, common_norm=False)
ax.set_ylabel("Fraction of Correlations");

Now that we are in a statistical testing mindset, a question you might have is that the correlations for cluster two tend to look much larger than the correlations for cluster one. How do we formalize this relationship? We can't use the fisher's exact test, since the fisher's exact test only worked for the case where our network was unweighted and the adjacency matrix took only binary values ($0$ or $1$). What can we do for a weighted network, when the adjacency matrix is not resticted to binary values? To formalize our situation a bit more, let's return to making some hypotheses about the data. We want to test whether the correlations in the second cluster exceed the correlations in the first cluster. This means that we have an alternative hypothesis $H_A:$ correrlations in cluster two $>$ correlations in cluster one. Remember that for two sample testing, the null hypothesis is going to be the opposite, so $H_0:$ correlations in cluster two $\leq $ correlations in cluster one. This is the weighted two-sample hypothesis testing problem.

##### The Mann-Whitney Wilcoxon $U$ Test

There are a variety of approaches to overcoming the weighted two-sample hypothesis testing problem. A popular approach is known as the two-sample $t$-test, which basically means that we assume that the samples are really just normally distributed, with some mean and variance, and then we forget about the actual data. We construct a test statistic by using properties about the normal distribution, not the actual data itself. To read more about the two-sample $t$-test, we would recommend you check out the [wikipedia article](https://en.wikipedia.org/wiki/Student%27s_t-test). Unforrtunately, if the data is not well-summarized by a normal distribution, the $t$-test tends to be a fairly poor choice for hypothesis testing. Instead, what we tend to use for the weighted two-sample hypothesis testing problem is we turn to what is known as the $U$-test. Instead of relying on assumptions that the data is normally distributed, the $U$-test does not assume anything about the distributions of the data samples. Even when the data is normally distributed, the $U$-test still tends to perform as well as the $t$-test. For this reason, we find the $U$-test to be a better procedure than other forms of testing that make heavy assumptions about the distributions of edge weights.

The $U$-test is very easy to implement in `graspologic`. We simply fit an SIEM to the data using the `SIEMEstimator` class:

In [ ]:
from graphbook_code import SIEMEstimator

model = SIEMEstimator()
model.fit(A, Z)

aand then we use the `compare` function to test whether cluster `2` is greater than cluster `1` by specifying `alternative="greater"` as a `methodarg`:

In [ ]:
from scipy.stats import mannwhitneyu

# perform mann-whitney u-test with alternative hypothesis that 
# correlations of cluster 2 > correlations of cluster 1
tstat, pval = model.compare(2, 1, method=mannwhitneyu, methodargs={"alternative": "greater"})
print("p-value: {:.3f}".format(pval))

Since the $p$-value is much less than $0.05$ (the print statement cut off at 3 decimal places, but in reality, it is about $10^{-64}$, which is *really* tiny!), with our decision threshold $\alpha = 0.05$, we accept the alternative hypothesis that the correlations in cluster two (the bilateral edges) really are bigger than the correlations in cluster one (the non-bilateral edges).

We could also use a similar, but slightly reworded, hypothesis and get the same answer. Note that saying that the correlations in cluster two are bigger than the correlations in cluster one would be the same as saying that the correlations in cluster one are smaller than the correlations in cluster two. We can reorder the clusters we are testing, and instead specify the alternative hypothesis as `"less"`:

In [ ]:
tstat, pval = model.compare(1, 2, method=mannwhitneyu, methodargs={"alternative": "less"})
print("p-value: {:.3f}".format(pval))

Which would obviously give us the same answer.

If you instead wanted to use a $t$-test or a different testing approach, you can do that very easily with graspologic, too. Just pass your testing strategy using the `method` argument, and pass additional arguments for the testing strategy to the `methodargs` dictionary. The only requirement is that the testing strategy takes two unnamed arguments, where the first argument is the first cluster of data, and the second argument is the second cluster of data. Here, we will be using `ttest_ind` [from scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html). We want to assume as little as possible about the data, so we will specify that the variances might not necessarily be equal as well:

In [ ]:
from scipy.stats import ttest_ind

# perform mann-whitney u-test with alternative hypothesis that 
# correlations of cluster 2 > correlations of cluster 1
tstat, pval = model.compare(2, 1, method=ttest_ind, methodargs={"alternative": "greater", "equal_var": False})
print("t statistic: {:.2f}, p-value: {:.3f}".format(tstat, pval))

And again we get a $p$-value that is much less than our decision threshold, so again we accept the alternative hypothesis that the correlations in cluster two exceed the correlations in cluster one.